In [5]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder \
    .appName("Read and Write Parquet to GCS") \
    .getOrCreate()

spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","../dtc-de-zoomcamp-410523-cc75fbfdb8b8.json")

In [16]:
# Specify the path to the Parquet files in GCS
bucket_name="mage-zoomcamp-jonah-oliver"
path=f"gs://{bucket_name}/ny_taxi_data/service=green/year=2019/month=*"

df=spark.read.parquet(path)

In [4]:
# Show the DataFrame
df.count()

6044050

In [5]:
df.registerTempTable('green')


/opt/conda/lib/python3.8/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2019-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
""")

In [11]:
path=f"gs://{bucket_name}/ny_taxi_data/service=yellow/year=2019/month=*"

df=spark.read.parquet(path)

In [12]:
df.registerTempTable('yellow')

In [13]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+-------------------+----+------+--------------+
|               hour|zone|amount|number_records|
+-------------------+----+------+--------------+
|2088-01-24 00:00:00|  24|  15.3|             1|
|2088-01-24 00:00:00|  41|   5.3|             1|
|2041-03-25 01:00:00| 151|  15.8|             1|
|2033-04-27 12:00:00| 107|   7.3|             1|
|2033-04-27 13:00:00| 263|   8.3|             1|
|2033-04-27 12:00:00| 229|  10.8|             1|
|2020-01-03 03:00:00|  97|  9.75|             1|
|2020-01-02 04:00:00| 100| 33.96|             1|
|2020-02-28 17:00:00| 237| 21.36|             1|
|2020-01-15 17:00:00| 233| 11.33|             1|
|2020-01-24 07:00:00| 164| 23.16|             1|
|2020-01-02 00:00:00| 163| 17.25|             1|
|2020-02-07 09:00:00| 233| 32.76|             1|
|2020-01-15 17:00:00| 263| 11.84|             1|
|2020-02-06 01:00:00| 263|  34.3|             1|
|2020-01-02 01:00:00| 164|   8.8|             1|
|2020-01-02 22:00:00| 158| 14.16|             1|
|2020-01-24 07:00:00

In [14]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2019-01-01 00:00:00|   3|             90.02|             2|
|2019-01-01 00:00:00|   7| 847.5499999999994|            68|
|2019-01-01 00:00:00|  11|              10.3|             1|
|2019-01-01 00:00:00|  14| 77.39999999999999|             3|
|2019-01-01 00:00:00|  16|50.599999999999994|             2|
|2019-01-01 00:00:00|  17|170.85000000000002|            10|
|2019-01-01 00:00:00|  18|             75.71|             3|
|2019-01-01 00:00:00|  20|             47.55|             1|
|2019-01-01 00:00:00|  24|              75.5|             6|
|2019-01-01 00:00:00|  25|1161.3399999999995|            48|
|2019-01-01 00:00:00|  26|              21.3|             1|
|2019-01-01 00:00:00|  28|             21.51|             1|
|2019-01-01 00:00:00|  29|              22.1|             2|
|2019-01-01 00:00:00|  3

In [15]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [48]:
bucket_name="mage-zoomcamp-jonah-oliver"
path=f"gs://{bucket_name}/ny_taxi_data/service=green/year=2019/report/revenue/"

df_green_revenue_tmp \
    .repartition(20) \
    .write.parquet(path, mode='overwrite')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+-------------+---------------------+
|               hour|zone|      green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-------------+---------------------+
|2019-01-01 00:00:00|  41| 1307.729999999998|                  97|         NULL|                 NULL|
|2019-01-01 00:00:00|  52|             18.96|                   1|         NULL|                 NULL|
|2019-01-01 00:00:00|  61| 330.2600000000001|                  17|         NULL|                 NULL|
|2019-01-01 00:00:00|  78|              34.9|                   3|         NULL|                 NULL|
|2019-01-01 00:00:00|  83|            147.78|                  12|         NULL|                 NULL|
|2019-01-01 00:00:00|  97| 708.4399999999999|                  32|         NULL|                 NULL|
|2019-01-01 00:00:00| 116|260.19000000000005|                  18|       

In [16]:
path=f"gs://{bucket_name}/ny_taxi_data/service=green/year=2019/report/revenue_total"

df_join.write.parquet(path, mode='overwrite')

24/03/05 15:56:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [7]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [8]:
path = f"gs://{bucket_name}/ny_taxi_data/zones/"
df.write.parquet(path, mode='overwrite')

In [18]:
path = f"gs://{bucket_name}/ny_taxi_data/zones/"
df_zones = spark.read.parquet(path)

In [19]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [25]:
df_result.show()

ConnectionRefusedError: [Errno 111] Connection refused

In [21]:
path = f"gs://{bucket_name}/ny_taxi_data/zones/report/revenue"

In [24]:
df_result.drop('LocationID', 'zone').write.parquet(path, mode='overwrite')

24/03/05 16:31:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [25]:
df_result.take(5)

[Row(hour=datetime.datetime(2019, 1, 1, 0, 0), zone=41, green_amount=1307.729999999998, green_number_records=97, yellow_amount=None, yellow_number_records=None, LocationID='41', Borough='Manhattan', Zone='Central Harlem', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2019, 1, 1, 0, 0), zone=52, green_amount=18.96, green_number_records=1, yellow_amount=None, yellow_number_records=None, LocationID='52', Borough='Brooklyn', Zone='Cobble Hill', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2019, 1, 1, 0, 0), zone=61, green_amount=330.2600000000001, green_number_records=17, yellow_amount=None, yellow_number_records=None, LocationID='61', Borough='Brooklyn', Zone='Crown Heights North', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2019, 1, 1, 0, 0), zone=78, green_amount=34.9, green_number_records=3, yellow_amount=None, yellow_number_records=None, LocationID='78', Borough='Bronx', Zone='East Tremont', service_zone='Boro Zone'),
 Row(hour=datetime.datetime(2019, 1, 